In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px

from PIL import Image
from os import listdir
from plotly.subplots import make_subplots
from plotly import graph_objs as go, offline
from plotly.express.colors import qualitative

# Allow figures to work in HTML-exported version of notebook.
offline.init_notebook_mode()

prefixes = [f"cv_{i}_" for i in range(1, 4)] + ["single_"]

## Two updates

12/05/21

This document discusses:
    
1. The pixelwise displacement and distance of our updater network's updates to its input.
2. The performance of Mahsa's approach: verifying her performance by running her code, and by performing the attack on her trained model using *our* code.

### A. Distance and displacement of updates

**Previous results**

Recall that there are four "configurations" of our model:
1. With an encoding network, and with z-score normalization of the input MNIST image.
2. With encoding, and without z-score normalization.
3. Without encoding, and with z-score normalization.
4. Without encoding, and without z-score normalization.

The original specification of the FGSM adversarial attack requires the input to be normalized to the 0-1 range for each pixel, and caps the perturbation of the input to keep it within this range.

With our approach, we see that **only one configuration: configuration 2**, has appreciable robustness to the FGSM attack. Note that in attacks for configurations with z-score normalization, the perturbation is **not capped**, to stay consistent with the distribution of input pixel values. 

In [ ]:
Image.open("images/robustness.png")

This corresponds with the configuration that does the most updating, in terms of magnitude of pixel value change/distance:

In [ ]:
Image.open("images/distance.png")

Specifically, we see that the average distance each pixel is updated stabilizes quite quickly for all configurations. **Moreover, it seems that the configuration which leads to the most updating is most robust against attack; this is intuitive, since perhaps this tendency also helps "undo" the FGSM adversarial perturbations most effectively.**

Let us look into this a little further, by considering how the average total *displacement* across pixels changes over updates and over epochs.

**Total displacement over epochs**

The purpose of considering average total pixelwise displacement, instead of just distance, is to verify that the updates aren't "meandering": that is, we want to consider whether the updater network is actually changing the value of pixels rather than frequently back-tracking on updates.





In [ ]:
def read_diagnostic(enc, std):
    enc_name = "encoder"
    std_name = "std"
    if not enc:
        enc_name = "no" + enc_name
    if not std:
        std_name = "no" + std_name
    curr_diagnostic = pd.read_csv(
        f"data/results/diagnostic_stats_{enc_name}_{std_name}_random_dense.csv"
    ).iloc[:, 1:]

    curr_diagnostic = curr_diagnostic.groupby("epoch")
    curr_diagnostic = curr_diagnostic.mean()

    config = ""
    if enc:
        config += "Enc. & "
    else:
        config += "No enc. & "
    if std:
        config += "std."
    else:
        config += "no std."
    curr_diagnostic["Config."] = config

    curr_diagnostic["Epoch"] = list(range(1, curr_diagnostic.shape[0] + 1))
    return curr_diagnostic

In [ ]:
all_diagnostic = None
for enc in [False, True]:
    for std in [False, True]:
        all_diagnostic = pd.concat(
            [all_diagnostic, read_diagnostic(enc, std)], ignore_index=True
        )

In [ ]:
fig = px.line(all_diagnostic, x="Epoch", y="mean_update_displacement", color="Config.")
fig.update_yaxes(title="Mean update displacement")
fig.update_layout(
    template="simple_white",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, x=0.15),
)

As can be seen, all configurations are efficient in their updating: the magnitude of the displacement of each pixel is similar to the distance it is moved by. Efficiency in this regard therefore cannot help us diagnose the the configuration's robustness. At most, we see that inclusion of an encoder into the network has a generally negative effect on pixel values, that needs to be corrected to set pixels to their 0/1 class positions.

**Total displacement over updates**

Recall that we give our updater network 50 iterations to update the input using the gradient of the network's output wrt the input, multiplied by a scaling factor of 0.1. Let us consider, for each of the configurations, the average cumulative displacement of each pixel over the 50 update iterations.

In [ ]:
def read_mvmt(enc, std):
    enc_name = "encoder"
    std_name = "std"
    if not enc:
        enc_name = "no" + enc_name
    if not std:
        std_name = "no" + std_name
    curr_mvmt = pd.read_csv(
        f"data/intra_mvmt/mvmt_{enc_name}_{std_name}_random_dense.csv"
    ).iloc[:, 1:]

    config = ""
    if enc:
        config += "Enc. & "
    else:
        config += "No enc. & "
    if std:
        config += "std."
    else:
        config += "no std."
    curr_mvmt["Config."] = config

    curr_mvmt["Update iteration"] = list(range(1, curr_mvmt.shape[0] + 1))
    return curr_mvmt

In [ ]:
all_mvmt = None
for enc in [False, True]:
    for std in [False, True]:
        all_mvmt = pd.concat([all_mvmt, read_mvmt(enc, std)], ignore_index=True)

In [ ]:
fig = px.line(all_mvmt, x="Update iteration", y=" Displacement", color="Config.")
fig.update_yaxes(title="Mean total update displacement")
fig.update_layout(
    template="simple_white",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, x=0.15),
)

Contrary to expectations, the updater networks from each configuration continue to update the input until the last iteration, albeit at different rates. On initial impression, this suggests that 50 updates may be insufficient: none of the configurations have learned to completely "level out" updating within 50 iterations.

Let us thus consider models trained with a 200-update iteration allowance:

In [ ]:
def read_mvmt_200(enc, std):
    enc_name = "encoder"
    std_name = "std"
    if not enc:
        enc_name = "no" + enc_name
    if not std:
        std_name = "no" + std_name
    curr_mvmt = pd.read_csv(
        f"data/intra_mvmt/mvmt_{enc_name}_{std_name}_random_dense_200.csv"
    ).iloc[:, 1:]

    config = ""
    if enc:
        config += "Enc. & "
    else:
        config += "No enc. & "
    if std:
        config += "std."
    else:
        config += "no std."
    curr_mvmt["Config."] = config

    curr_mvmt["Update iteration"] = list(range(1, curr_mvmt.shape[0] + 1))
    return curr_mvmt

In [ ]:
all_mvmt_200 = None
for enc in [False, True]:
    for std in [False, True]:
        all_mvmt_200 = pd.concat(
            [all_mvmt_200, read_mvmt_200(enc, std)], ignore_index=True
        )

In [ ]:
fig = px.line(all_mvmt_200, x="Update iteration", y=" Displacement", color="Config.")
fig.update_yaxes(title="Mean total update displacement")
fig.update_layout(
    template="simple_white",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, x=0.15),
)

When we allow the model 200 update iterations:

1. The configurations of the network without the encoder learn to change pixel values by about the same total amount, just more slowly. For them, it seems 50 updates were roughly sufficient.
2. Configurations with the encoder continue to make approximately the same kinds of updates, leading to a significantly larger total after 200 update iterations.

Despite these differences, the performance improvement of the model under each configuration is at least slightly worse than when they were allowed fewer updates:

In [ ]:
def read_attack(enc, std, new):
    enc_name = "encoder"
    std_name = "std"
    new_name = "" if not new else "_200"
    if not enc:
        enc_name = "no" + enc_name
    if not std:
        std_name = "no" + std_name
    curr_results = pd.read_csv(
        f"data/results/robust_attack_results_{enc_name}_{std_name}_random_dense{new_name}.csv"
    ).iloc[:, 1:]

    config = ""
    if enc:
        config += "Enc. & "
    else:
        config += "No enc. & "
    if std:
        config += "std."
    else:
        config += "no std."
    curr_results["Config."] = config

    return curr_results

In [ ]:
def read_improvement(enc, std):
    attack_200 = read_attack(enc, std, True)
    attack_50 = read_attack(enc, std, False)
    attack_200["Accuracy improvement"] = attack_200.accuracy - attack_50.accuracy
    attack_200["Epsilon"] = attack_200.epsilon
    return attack_200

In [ ]:
all_improvement = None
for enc in [False, True]:
    for std in [False, True]:
        all_improvement = pd.concat(
            [all_improvement, read_improvement(enc, std)], ignore_index=True
        )

In [ ]:
fig = px.line(all_improvement, x="Epsilon", y="Accuracy improvement", color="Config.")
fig.update_layout(
    template="simple_white",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, x=0.15),
)

The reduction in performance under all configurations suggests our updater network training is ineffective.

Specifically, the slightly worse performance of the 200-update model under our best configuration `Enc. & std.`, coupled with the fact that the model does not learn to level out updates within 200 epochs despite this worse performance, suggests that the training of our updater network is divorced from the model's robustness. This reinforces a statistic we discussed in the past: training MSE of our updater network under the `Enc. & std.` configuration is about 50 times greater than in configurations without encoders, even if the resulting model is far more robust.

Moreover, there appear to be problems with the nature of the updater network surface itself: for the configurations with no encoder, we see that more (but smaller) updates result in significantly worse performance. Note that the zero accuracy improvement for greater values of the attack epsilon is due to the accuracy of both models - the original 50-update model and the new 200-update model - being zero under these attacks. Does the surface lack smoothness?

**Given these issues, our primary question to answer is: how can we improve the training of our updater network?**

### B. Performance of Mahsa's approach

Here, we verify the results of Mahsa's approach.

First, let us consider the performance of Mahsa's trained model using her `foolbox` library-based attack code:

In [ ]:
robust_accuracy = [
    1.0,
    0.9979730414513023,
    0.9960474308300395,
    0.9936150805716023,
    0.9916894699503395,
    0.9887503800547279,
    0.9857099422316814,
    0.9820614168440256,
    0.9767913246174116,
    0.967670011148272,
    0.951961082395865,
    0.9294618425053207,
    0.8923685010641532,
    0.8224384311340833,
    0.7124759298672343,
]

her_accuracy = [
    1.0,
    0.99,
    0.985,
    0.98,
    0.975,
    0.97,
    0.965,
    0.96,
    0.94,
    0.92,
    0.90,
    0.86,
    0.805,
    0.72,
    0.63,
]

mahsa_df_orig = pd.concat(
    [
        pd.DataFrame.from_dict(
            {
                "Attack epsilon": np.round(np.arange(0.0, 0.75, 0.05), 2).tolist(),
                "Accuracy": robust_accuracy,
                "Source": "Replicated results",
            }
        ),
        pd.DataFrame.from_dict(
            {
                "Attack epsilon": np.round(np.arange(0.0, 0.75, 0.05), 2).tolist(),
                "Accuracy": her_accuracy,
                "Source": "Paper results",
            },
        ),
    ],
    ignore_index=True,
)

fig = px.line(mahsa_df_orig, x="Attack epsilon", y="Accuracy", color="Source")
fig.update_layout(
    template="simple_white",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, x=0.3),
)

I had access to a trained model under one configuration of hyperparameters (which used only her regularization), and the results under attack are reasonably similar to the reported results of the model under a similar set of hyperparameters.

As previously stated, Mahsa z-score normalizes her inputs, which changes the range of each pixel from \[0, 1\] to ($-\infty$, $\infty$).

However, her FGSM attack, implemented using the `foolbox` library, constrains the perturbed input's pixels within the \[0, 1\] bound.

The following plot displays the performance of her model on an unconstrained attack: the same one executed on our best-performing `Enc. & std.` configuration:

In [ ]:
foolbox_acc = [
    1.0,
    0.9979730414513023,
    0.9960474308300395,
    0.9936150805716023,
    0.9916894699503395,
    0.9887503800547279,
    0.9857099422316814,
    0.9820614168440256,
    0.9767913246174116,
    0.967670011148272,
    0.951961082395865,
    0.9294618425053207,
    0.8923685010641532,
    0.8224384311340833,
    0.7124759298672343,
]

manual_acc = [
    0.990,
    0.980,
    0.969,
    0.953,
    0.938,
    0.914,
    0.883,
    0.852,
    0.806,
    0.749,
    0.679,
    0.604,
    0.517,
    0.448,
    0.374,
]

our_acc = [
    0.986,
    0.969,
    0.925,
    0.884,
    0.818,
    0.717,
    0.606,
    0.476,
    0.375,
    0.269,
    0.180,
    0.115,
    0.082,
    0.060,
    0.043,
]

mahsa_df_orig = pd.concat(
    [
        pd.DataFrame.from_dict(
            {
                "Attack epsilon": np.round(np.arange(0.0, 0.75, 0.05), 2).tolist(),
                "Accuracy": foolbox_acc,
                "Source": "Mahsa (constrained attack)",
            }
        ),
        pd.DataFrame.from_dict(
            {
                "Attack epsilon": np.round(np.arange(0.0, 0.75, 0.05), 2).tolist(),
                "Accuracy": manual_acc,
                "Source": "Mahsa (unconstrained attack)",
            },
        ),
        pd.DataFrame.from_dict(
            {
                "Attack epsilon": np.round(np.arange(0.0, 0.75, 0.05), 2).tolist(),
                "Accuracy": our_acc,
                "Source": "Updater network (enc. & std.)",
            },
        ),
    ],
    ignore_index=True,
)

fig = px.line(mahsa_df_orig, x="Attack epsilon", y="Accuracy", color="Source")
fig.update_layout(
    template="simple_white",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, x=0.1),
)

We see that while removing the constraint on the perturbed input degrades performance somewhat, it is still far superior to our best configuration.

Moreover, a key drawback of our model is its slow evaluation speed due to the update process; our model is about 40 times slower in evaluation than her's.